# <center> International Maritime Piracy </center> 


This is part of my EDA process along what I was thinking. This next field (The What, the How) contains the remainder of the EDA questions I have. The code serves only as example as it is contained it /src folder

#### The What

* __Basic Location & Time Trends__
    * What are the year on year trends? 
    * When during the year are attacks most common?
    * Are there any days of the weeks where attaks don't occur (e.g. Friday in MENA or Sunday in Catholic areas)?
 
* __Via Text__
    * How many kidnappings were there?
    * How many murders or killings were there?
    * What about attackers? How may of there were killed?

* __Inland/Offshore__
    * Is it possible to know, using a mask of land coverage, how far from shore attacks occur?
    * What about ones that are more than a mile or so?
    
* __Statistical Analysis__
    * Would like to 
 
#### The How  
 
* __Mapping the Attacks__
    * Need either something with a good API or use something like Folium  
    * (NavArea or SubReg overlay)
    
* __Other Goals__
    * Containerized Web App (AWS, Azure or GCP -- possibly even Heroku or Linode if free)
    

In [ ]:
import pandas as pd
import re

In [ ]:
data = pd.read_csv('file.csv')

### See if pandas will read the csv without creating the second line and see if anything is odd.

In [3]:
data.head(5)

,reference,date,position,navArea,subreg,hostility,victim,description,Unnamed: 8
0,2019-71,2019-07-21,"2°55'00""N \n105°10'00""E",XI,71,PIRATES,BULK CARRIER,"INDONESIA: On 21 July, seven pirates armed wit...",NaN
1,2019-70,2019-07-21,"8°28'00""N \n13°28'00""W",II,51,ROBBERS,CONTAINER SHIP,"SIERRA LEONE: On 21 July, five robbers armed w...",NaN
2,2019-68,2019-07-16,"1°10'23""N \n103°39'09""E",XI,71,ROBBERS,TUG,"INDONESIA: On 16 July, robbers boarded a barge...",NaN
3,2019-69,2019-07-13,"9°35'15""N \n79°28'19""W",IV,26,PIRATES,PLEASURE CRAFT,"PANAMA: On 13 July, a monohull with two person...",NaN
4,2019-67,2019-07-13,"2°58'00""N \n4°40'00""E",II,57,PIRATES,CARGO SHIP,"NIGERIA: On 13 July, pirates boarded the Turke...",NaN


### Unnamed: 8 is odd. Is it all NaNs or something else? 

In [4]:
data['Unnamed: 8'].unique()

array([nan])

In [5]:
data.drop(['Unnamed: 8'], axis=1, inplace=True)

In [6]:
data.tail(2)

,reference,date,position,navArea,subreg,hostility,victim,description
7836,1985-2,1978-05-16,"7°59'60""N \n119°59'60""E",XI,92,PIRATES,MERCHANT VESSEL,"SULU SEA: Lloyd's List of 15 May reports a 5,0..."
7837,1989-12,1978-05-01,"5°59'60""N \n119°30'00""E",XI,92,UNKNOWN,UNKNOWN VESSEL,SOUTH SULU SEA ...


### Looks like there are 7825 attacks with all the fields filled out. 

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7838 entries, 0 to 7837
Data columns (total 8 columns):
reference      7838 non-null object
date           7838 non-null object
position       7838 non-null object
navArea        7835 non-null object
subreg         7838 non-null int64
hostility      7829 non-null object
victim         7825 non-null object
description    7838 non-null object
dtypes: int64(1), object(7)
memory usage: 490.0+ KB


### Split postion into Lat/Lon in prep for conversion to decimal degrees

In [8]:
# data['position'].str.partition('\n')[[2, 2]].rename({2: 'lat', 3: 'loing'}, axis=1)
# data['Latitude'], data['Longitude'] = data['position'].str.split("\n", expand=True)
# data.join(data['position'].str.split('\n', 1, expand=True).rename(columns={0:'Lat', 1:'Long'}))

# above is part of the graveyard of ineffective pandas code

data = data.join(data['position'].str.split(expand=True)).rename(columns={0:'Latitude', 1:'Longitude'})
data.drop('position', axis=1, inplace=True)


In [9]:
def dms2dd(s):
    # example: s = """0°51'56.29"S"""
    degrees, minutes, seconds, direction = re.split('[°\'"]+', s)
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S','W'):
        dd*= -1
    return dd

In [10]:
data['Latitude'] = data['Latitude'].apply(dms2dd)
data['Longitude'] = data['Longitude'].apply(dms2dd)

In [11]:
data.head(2)

,reference,date,navArea,subreg,hostility,victim,description,Latitude,Longitude
0,2019-71,2019-07-21,XI,71,PIRATES,BULK CARRIER,"INDONESIA: On 21 July, seven pirates armed wit...",2.916667,105.166667
1,2019-70,2019-07-21,II,51,ROBBERS,CONTAINER SHIP,"SIERRA LEONE: On 21 July, five robbers armed w...",8.466667,-13.466667


### Under 'hostility' - how many different terms are there? 323

In [12]:
data['hostility'].nunique()
#data['hostility'].unique()

323

### Under 'victim' - how many different terms are there? 1223

In [13]:
data['victim'].nunique()
data['hostility'].unique();